In [2]:
!pip3 install tomlkit
!pip3 install -U neo4j

In [3]:
from neo4j import GraphDatabase

database_name = "booksnyt"
username = "neo4j"
password = "12345678"
uri = "neo4j://127.0.0.1:7687"

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")

Successfully connected to Neo4j!


In [4]:
import pandas as pd

df = pd.read_json("books_historical (1).json")

In [5]:
# Parse dates
df["published_date"] = pd.to_datetime(df["published_date"])
df["bestsellers_date"] = pd.to_datetime(df["bestsellers_date"])


# Helper: month -> season
def month_to_season(m):
    if m in [12, 1, 2]:
        return "Winter"
    if m in [3, 4, 5]:
        return "Spring"
    if m in [6, 7, 8]:
        return "Summer"
    return "Fall"


df["season"] = df["published_date"].dt.month.map(month_to_season)

# Convert dates to string for Neo4j
df["published_date_str"] = df["published_date"].dt.date.astype(str)
df["bestsellers_date_str"] = df["bestsellers_date"].dt.date.astype(str)

df[["title", "author", "publisher", "published_date_str", "bestsellers_date_str", "season"]].head()


,title,author,publisher,published_date_str,bestsellers_date_str,season
0,THE LOST SYMBOL,Dan Brown,Knopf Doubleday,2009-12-27,2009-12-12,Winter
1,U IS FOR UNDERTOW,Sue Grafton,Putnam,2009-12-27,2009-12-12,Winter
2,CROSS FIRE,James Patterson,"Little, Brown",2009-12-27,2009-12-12,Winter
3,"UNDER THE DOME, PART 1",Stephen King,Scribner,2009-12-27,2009-12-12,Winter
4,THE HELP,Kathryn Stockett,Amy Einhorn/Putnam,2009-12-27,2009-12-12,Winter


In [6]:
# Columns we care about for the graph
cols_for_neo4j = [
    "primary_isbn13",
    "title",
    "author",
    "publisher",
    "list_name",
    "bestsellers_date_str",
    "published_date_str",
    "rank",
    "weeks_on_list",
    "season",
]

df_export = df[cols_for_neo4j].copy()

# Rename columns to nicer names
df_export = df_export.rename(columns={
    "primary_isbn13": "isbn13",
    "bestsellers_date_str": "bestsellers_date",
    "published_date_str": "published_date"
})

# Only require rank to be present (DON'T filter on isbn13)
df_export = df_export.dropna(subset=["rank"])

print("Rows going to Neo4j:", df_export.shape[0])
df_export.head()


Rows going to Neo4j: 5080


,isbn13,title,author,publisher,list_name,bestsellers_date,published_date,rank,weeks_on_list,season
0,9780385533133,THE LOST SYMBOL,Dan Brown,Knopf Doubleday,hardcover-fiction,2009-12-12,2009-12-27,1,13,Winter
1,9780399155970,U IS FOR UNDERTOW,Sue Grafton,Putnam,hardcover-fiction,2009-12-12,2009-12-27,2,2,Winter
2,9780316071901,CROSS FIRE,James Patterson,"Little, Brown",hardcover-fiction,2009-12-12,2009-12-27,3,4,Winter
3,9781439148501,"UNDER THE DOME, PART 1",Stephen King,Scribner,hardcover-fiction,2009-12-12,2009-12-27,4,5,Winter
4,9780399155345,THE HELP,Kathryn Stockett,Amy Einhorn/Putnam,hardcover-fiction,2009-12-12,2009-12-27,5,37,Winter


In [7]:
#define constraints
def create_constraints(driver, database_name):
    queries = [
        """
        CREATE CONSTRAINT book_isbn IF NOT EXISTS
        FOR (b:Book)
        REQUIRE b.isbn13 IS UNIQUE
        """,
        """
        CREATE CONSTRAINT author_name IF NOT EXISTS
        FOR (a:Author)
        REQUIRE a.name IS UNIQUE
        """,
        """
        CREATE CONSTRAINT publisher_name IF NOT EXISTS
        FOR (p:Publisher)
        REQUIRE p.name IS UNIQUE
        """,
        """
        CREATE CONSTRAINT listweek_unique IF NOT EXISTS
        FOR (lw:ListWeek)
        REQUIRE (lw.list_name, lw.bestsellers_date) IS UNIQUE
        """,
        """
        CREATE CONSTRAINT season_name IF NOT EXISTS
        FOR (s:Season)
        REQUIRE s.name IS UNIQUE
        """
    ]
    with driver.session(database=database_name) as sess:
        for q in queries:
            sess.run(q)


create_constraints(driver, database_name)
print("Constraints created (or already existed).")


Constraints created (or already existed).


In [8]:
load_query = """
MERGE (b:Book {isbn13: $isbn13})
ON CREATE SET
    b.title = $title,
    b.published_date = $published_date

MERGE (a:Author {name: $author})
MERGE (a)-[:WROTE]->(b)

MERGE (p:Publisher {name: $publisher})
MERGE (p)-[:PUBLISHED]->(b)

MERGE (lw:ListWeek {
    list_name: $list_name,
    bestsellers_date: $bestsellers_date
})

MERGE (b)-[r:APPEARED_ON]->(lw)
ON CREATE SET
    r.rank = $rank,
    r.weeks_on_list = $weeks_on_list

MERGE (s:Season {name: $season})
MERGE (b)-[:PUBLISHED_IN]->(s)
"""


In [9]:
import math
import numpy as np


def send_row_to_neo4j(tx, row):
    # convert pandas types → plain Python
    rank = int(row["rank"]) if not pd.isna(row["rank"]) else None
    weeks = int(row["weeks_on_list"]) if not pd.isna(row["weeks_on_list"]) else None

    # isbn13 can be missing; send None in that case
    isbn_value = None if pd.isna(row["isbn13"]) else str(row["isbn13"])

    tx.run(
        load_query,
        isbn13=isbn_value,
        title=str(row["title"]),
        author=str(row["author"]),
        publisher=str(row["publisher"]),
        list_name=str(row["list_name"]),
        bestsellers_date=str(row["bestsellers_date"]),
        published_date=str(row["published_date"]),
        rank=rank,
        weeks_on_list=weeks,
        season=str(row["season"]),
    )


In [10]:
with driver.session(database=database_name) as sess:
    sess.run("MATCH (n) DETACH DELETE n")
print("Cleared all nodes/relationships in booksnyt.")


Cleared all nodes/relationships in booksnyt.


In [11]:
from tqdm import tqdm  # pip install tqdm if you don't have it

with driver.session(database=database_name) as write_session:
    for _, row in tqdm(df_export.iterrows(), total=len(df_export)):
        write_session.execute_write(send_row_to_neo4j, row)

print("Finished loading data into Neo4j!")


100%|██████████| 5080/5080 [00:45<00:00, 110.89it/s]

Finished loading data into Neo4j!


ListWeek shows which NYT list the book appeared on

the queries i have run so far

(schema view)
CALL db.schema.visualization();
→ Explains the data model to your audience.

(book + author + publisher + season)
MATCH path = (b:Book)-[:WROTE|:PUBLISHED|:PUBLISHED_IN|:APPEARED_ON]->(x)
RETURN path
LIMIT 200;
→ Shows how every bestseller lives inside a web of authors, publishers, weeks, and seasons.

may run more queries to help sasha and abby answer their questions/ provide visuals 